In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
from crowdnalysis.cmdstan import StanMultinomialOptimizeConsensus, StanMultinomial2OptimizeConsensus, StanDSOptimizeConsensus
from crowdnalysis.dawid_skene import DawidSkene
from crowdnalysis.majority_voting import MajorityVoting
import numpy as np
import json

In [71]:
mv = MajorityVoting()
smoc = StanMultinomialOptimizeConsensus()
sm2oc = StanMultinomial2OptimizeConsensus()
sdsoc = StanDSOptimizeConsensus()
ds = DawidSkene()

In [64]:
t = 300
num_annotations_per_task = 50
real_tau = np.array([0.1, 0.3, 0.6])
k = len(real_tau)
real_pi = np.array([[0.5, 0.3, 0.2], [0.2, 0.6, 0.2], [0.25, 0.3, 0.45]]) 
print(real_pi)
t_C, m = smoc.sample(t, num_annotations_per_task, parameters = {'tau':real_tau, 'pi':real_pi})
print(t_C)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_tasks
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_annotations
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


[[0.5  0.3  0.2 ]
 [0.2  0.6  0.2 ]
 [0.25 0.3  0.45]]
[1 2 2 2 2 2 2 2 1 1 0 2 2 1 0 1 2 2 2 2 2 2 1 2 1 1 2 1 2 0 2 2 2 0 2 2 2
 2 1 1 2 2 2 0 2 2 0 2 1 2 1 2 2 1 1 1 2 2 2 2 1 2 2 2 1 2 2 1 1 2 2 0 2 2
 2 1 2 1 2 1 1 1 2 2 2 2 0 1 1 0 1 2 1 0 1 1 2 2 1 1 2 1 2 1 2 2 1 2 2 0 1
 2 2 1 1 2 1 1 2 2 0 0 2 1 1 2 1 1 2 2 2 2 2 2 2 2 1 2 1 2 0 2 0 0 1 2 2 2
 2 2 2 1 0 2 1 2 0 2 2 2 2 2 2 0 2 2 2 2 0 2 2 2 0 1 0 2 1 2 2 2 2 2 2 0 2
 2 2 2 2 1 0 2 1 2 2 1 2 2 2 2 0 2 2 2 2 2 2 2 1 1 2 2 2 2 0 2 1 2 2 2 2 1
 1 1 2 2 0 2 2 1 0 1 2 2 1 2 1 2 1 1 2 2 1 0 2 2 1 2 2 2 2 2 1 1 2 1 1 1 1
 2 1 2 2 1 2 2 2 1 2 1 2 2 2 2 0 1 2 0 1 0 2 2 2 2 2 2 2 2 2 1 2 1 1 2 2 2
 2 2 2 1]


In [65]:
m

array([[  0,   0,   1],
       [  0,   0,   0],
       [  0,   0,   0],
       ...,
       [299,   0,   1],
       [299,   0,   1],
       [299,   0,   0]])

In [66]:
t_A = m[:,0]+1
w_A = m[:,1]
ann = m[:,2]+1
w = 1
#t = np.unique(t_A).shape[0]
a = m.shape[0]
json_data = {'w': w,
             't': t,
             'a': a,
             'k': k,
             't_A': t_A.tolist(),
             'w_A': w_A.tolist(),
             'ann': ann.tolist()}

In [67]:
print("Max w_A:",np.max(w_A))
with open('multinomial.json', 'w') as outfile:
    json.dump(json_data, outfile, indent=2)

Max w_A: 0


In [68]:
def print_vars(d,v):
    for name in v:
        if name in d:
            print(name,"=",d[name])

In [69]:
def err_rate(t_C, t_C_m):
    max_t_C_m = np.argmax(t_C_m,axis=1)
    #print(max_t_C_m)
    return 1-(np.sum(t_C==max_t_C_m)/len(t_C))
def log_score(t_C, t_C_m):
    f = t_C_m.flatten()
    start = k*np.arange(t)
    return np.sum(np.log(f[start+t_C]))

In [72]:
print("Real parameters")
print("tau=",real_tau)
print("pi=",real_pi)
for model in [mv, smoc, ds, sm2oc, sdsoc]:
    print("*********")
    print(model.name)
    t_C_m, d = model.m_fit_and_compute_consensus(m, t, k, w)
    print("error_rate:", err_rate(t_C, t_C_m))
    print("log_score:", log_score(t_C, t_C_m))
    #print(d.keys())
    print_vars(d,["p","_pi","tau","pi"])
    #if "p" in d:
    #    print (d['p'],d['pi_'])
    #else:
    #    print(d['tau'],d['pi'])

/home/cerquide/prj/crowdnalysis/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/home/cerquide/prj/crowdnalysis/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial2.fit_and_consensus
INFO:cmdstanpy:start chain 1


Real parameters
tau= [0.1 0.3 0.6]
pi= [[0.5  0.3  0.2 ]
 [0.2  0.6  0.2 ]
 [0.25 0.3  0.45]]
*********
MajorityVoting
error_rate: 0.09666666666666668
log_score: -inf
*********
StanMultinomialOptimize
error_rate: 0.9833333333333333
log_score: -3158.8955345157665
tau = [0.28604  0.602819 0.111141]
pi = [[0.208726 0.594395 0.196879]
 [0.241121 0.296014 0.462865]
 [0.485923 0.298027 0.21605 ]]
*********
DawidSkene
error_rate: 0.023333333333333317
log_score: -24.027093563313812
tau = [0.11049128 0.28735918 0.60214954]
pi = [[[0.48935139 0.30020856 0.21044006]
  [0.20688674 0.59566061 0.19745266]
  [0.24162651 0.29522903 0.46314446]]]
*********
StanMultinomial2Optimize


INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:compiling stan program, exe file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/DS.fit_and_consensus
INFO:cmdstanpy:compiler options: stanc_options={'include_paths': ['/home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan']}, cpp_options=None


error_rate: 0.023333333333333317
log_score: -24.01264230470225
tau = [0.108245 0.288061 0.603694]
pi = [[0.489968 0.300028 0.210004]
 [0.206837 0.595814 0.197348]
 [0.241638 0.295225 0.463137]]
*********
StanDSOptimize


INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/DS.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


error_rate: 0.9833333333333333
log_score: -3158.8955345157665
tau = [0.28604  0.602819 0.111141]
pi = [[[0.208726 0.594395 0.196879]
  [0.241121 0.296014 0.462865]
  [0.485923 0.298027 0.21605 ]]]


In [35]:
t_C, d = smoc.m_fit_and_compute_consensus(m, t, k, w)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'tau': array([0.994216  , 0.00329214, 0.00249192]),
 'pi': array([[2.51233e-01, 3.98604e-01, 3.50163e-01],
        [4.85310e-05, 9.99946e-01, 5.33141e-06],
        [2.51040e-08, 1.21125e-10, 1.00000e+00]]),
 'log_p_t_C': array([[  -54.3847,  -392.038 ,  -581.345 ],
        [  -51.5747,  -228.678 ,  -848.523 ],
        [  -53.6478,  -422.948 ,  -483.176 ],
        [  -55.7128,  -383.204 ,  -651.346 ],
        [  -54.3847,  -392.038 ,  -581.345 ],
        [  -51.9067,  -226.47  ,  -866.023 ],
        [  -53.5911,  -384.314 ,  -569.178 ],
        [  -61.2114,  -398.643 ,  -822.512 ],
        [  -58.7739,  -336.835 ,  -872.016 ],
        [  -50.9107,  -233.096 ,  -813.522 ],
        [  -54.7167,  -389.83  ,  -598.845 ],
        [  -54.0851,  -263.994 ,  -862.187 ],
        [  -57.7454,  -473.713 ,  -521.173 ],
        [  -49.1372,  -166.87  ,  -898.027 ],
        [  -55.3808,  -385.413 ,  -633.845 ],
        [  -55.0326,  -452.748 ,  -467.174 ],
        [  -54.1823,  -406.389 ,  -541.01  

In [ ]:
m[:,1]=0
a = ds.m_fit_and_compute_consensus(m, t, k, 1)
a


In [ ]:
d = smoc.m_fit(m, t, k, w, t_C)
d

In [ ]:
d2 = smoc.m_compute_consensus(m, t, k, w, data=d)
d2